# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [10]:
!pip install yahoo_fin
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 5.0 MB/s 


In [11]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter library for 
import math #The Python math module
import yahoo_fin.stock_info as si

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [26]:
#stocks = pd.read_csv('sample_data/sp_500_stocks.csv')

In [16]:
lt = si.tickers_sp500()
stocks = pd.DataFrame()
stocks['Ticker']=lt
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [2]:
#from secrets import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [3]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 121492474,
 'calculationPrice': 'close',
 'change': 5.15,
 'changePercent': 0.02959,
 'close': 0,
 'closeSource': 'lcioiffa',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 183.18,
 'iexCloseTime': 1698635155615,
 'iexLastUpdated': 1720222169424,
 'iexMarketPercent': 12.71921765840402,
 'iexOpen': 176.146,
 'iexOpenTime': 1689519885316,
 'iexRealtimePrice': 181.61,
 'iexRealtimeSize': 5,
 'iexVolume': 1447183,
 'isUSMarketOpen': False,
 'lastTradeTime': 1656386399415,
 'latestPrice': 183.2,
 'latestSource': 'Close',
 'latestTime': 'December 15, 2021',
 'latestUpdate': 1649065269743,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
data['latestPrice'] , data['marketCap']

(183.2, 3075107682762)

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [19]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [27]:
for symbol in stocks['Ticker']:
  api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(api_url).json()
  print (symbol, data['latestPrice'])
  final_dataframe.append({'Ticker': symbol, 'Price' : data['latestPrice'] , 'Market Capitalization' : data['marketCap'], 'Number Of Shares to Buy' : 0}, ignore_index=True)
final_dataframe  

A 156.07
AAL 17.58
AAP 245.76
AAPL 180.3
ABBV 135.06
ABC 129.65
ABMD 327.96
ABT 136.99
ACN 383.2
ADBE 659.77
ADI 184.41
ADM 67.57
ADP 243.61
ADSK 273.18
AEE 90.32
AEP 89.34
AES 25.11
AFL 58.06
AIG 55.36
AIV 8.05
AIZ 157.46
AJG 174.04
AKAM 120.39
ALB 239.9
ALGN 610.57
ALK 51.45
ALL 116.42
ALLE 134.06
ALXN 189.9
AMAT 153.72
AMCR 12.17
AMD 146.8
AME 146.19
AMGN 227.74
AMP 301.87
AMT 287.49
AMZN 3549.6
ANET 145.38
ANSS 416.28
ANTM 463.3
AON 299.09
AOS 85.07
APA 25.5
APD 307.13
APH 86.56
APTV 166.94
ARE 229.48
ATO 99.03
ATVI 62.41
AVB 256.3
AVGO 642.23
AVY 219.47
AWK 187.33
AXP 165.75
AZO 2062.93
BA 203.73
BAC 44.9
BAX 86.7
BBY 104.14
BDX 252.59
BEN 34.93
BF.B 75.29
BIIB 243.14
BIO 771.59
BK 58.9
BKNG 2138.12
BKR 24.65
BLK 923.86
BLL 92.21
BMY 62.84
BR 180.44
BRK.B 303.77
BSX 40.58
BWA 45.55
BXP 118.38
C 62.35
CAG 33.63
CAH 49.3
CARR 56.19
CAT 207.12
CB 196.05
CBOE 130.66
CBRE 108.63
CCI 199
CCL 18.74
CDNS 186.16
CDW 201.31
CE 164.38
CERN 79.25
CF 65.05
CFG 47.59
CHD 100.7
CHRW 107.96
CHTR 

JSONDecodeError: ignored

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: